In [6]:
class ExtractMeasurements:
    def __init__(self,c8y,source):
        
        from datetime import timedelta
        import pandas as pd

        #instância necessárias para realizar as operaçoes na API
        self.c8y = c8y
        self.source = source

        self.delta_twohours = timedelta(hours=2) # 2 hours older
        self.delta_onehour = timedelta(hours=1) # 1 hour older


    #classe para extrair de devices novos
    def extract_new_devices(self) -> list:
        def tratamento(_hour_measu):
            df = {}
            for i,v in enumerate(_hour_measu):
                fragments = v.fragments # caracteristicas do dados puxado. Daí tiramos as propriedes que queremos
                col_name = list(fragments.keys())[0] # nome da nossa coluna no df
                aux = list(fragments[col_name].keys())[0] # parametro utilizado para entrar no aninhamento
                value = [list(fragments[col_name][aux].values())[1]] #valor do parametro atual -> inserimos ele numa lista para poder appendar no script abaixo

                # no primeiro loop, precisamos definir as colunas do dataframe, nos próximos basta appendarmos.
                try:
                    df[col_name].append(value[0])
                except KeyError as error:
                    df[col_name] = value

                comp_min = min(map(len,df.values())) # alguns parametros possuem time interval de envio de dados diferentes. Isso faz com que seja necessario equilibrarmos ele
                #limitando as linhas do dataframe
                for k,v in df.items():
                    df[k] = v[:comp_min]
                
                df = pd.DataFrame(df)
                return df
        
        one_hour_measu = self.c8y.measurements.select(source=self.source,max_age=self.delta_onehour)
        
        two_hour_measu = self.c8y.measurements.select(source=self.source,max_age=self.delta_twohours, min_age=self.delta_onehour)
        
        return tratamento(one_hour_measu)


    def extract_old_devices(self):
        pass
    
        
        

# TESTE DA CLASSE

In [7]:
# libs necessárias
from c8y_api import CumulocityApi
from datetime import timedelta
import pandas as pd 

import json
import os

config_path = os.path.join(os.getcwd(), '..','..','','configs','config.json')
with open(config_path) as config_file:
    login = json.load(config_file)

# instâncias gloabais
c8y = CumulocityApi(
    base_url= login['url'],
    tenant_id= login['tenant'],
    username= login['username'],
    password= login['password'],
)

#devices de teste
# 621151045972 -> unilever || 1069740447 -> serbom || 611284292311 -> garoto cag 09 || 481304022994 -> garoto cag 10
sources = ['621151045972', '1069740447', '611284292311','481304022994']


In [8]:
extract = ExtractMeasurements(c8y=c8y,source=sources[2])

In [9]:
extract.extract_new_devices()

[   EB_CAG09
 0  8.347283,
    EB_CAG09
 0  7.218605]